### __Data Preprocessing: Regression Model v3__

In [1]:
# import libraries
import numpy as np
import pandas as pd
import random
from datetime import datetime, timezone, timedelta
import datetime as dt
import time

# import matplotlib
from matplotlib import pyplot
import matplotlib.pyplot as plt

# to be able to see multiple ouputs from sungle cell
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

#### __Preprocessing Data Steps__
> - Step 1. Remove insignificant variables:
    - string variables: branded_code_share, mkt_carrier, tail_num, origin, origin_city_name, dest, dest_city_name, dup, flights
    - variables with high correlation: mkt_carrier_fl_num, op_carrier_fl_num, distance (highly correlated to crs_elapsed_time)
> - Step 2. Create additional features: 
    - year, month and day_of_week from 'fl_date'
    - hours from both, 'crs_dep_time' and 'crs_arr_time'
> - Step 3. Get features from passengers and fuel
    - from passengers_summary: speed, passengers_mean, and passengers_sum
    - from fuel_summary: total_cost mean and sum, and total_gallons mean and sum
> - Setp 4. Create dummy variables based on the following features:
    - year
    - month
    - day_of_week
    - mkt_unique_carrier & op_unique_carrier
    - origin_airport_id & dest_airport_id
    - two hours variables (from 'crs_dep_time' and 'crs_arr_time')

-------------------------------------------------

#### __Import Datasets__

In [2]:
# Import training csv file
df = pd.read_csv('reduced_train_df_56kv2.csv')
df.info()
df.head(3)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55754 entries, 0 to 55753
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fl_date             55754 non-null  object 
 1   mkt_unique_carrier  55754 non-null  object 
 2   branded_code_share  55754 non-null  object 
 3   mkt_carrier         55754 non-null  object 
 4   mkt_carrier_fl_num  55754 non-null  int64  
 5   op_unique_carrier   55754 non-null  object 
 6   tail_num            55754 non-null  object 
 7   op_carrier_fl_num   55754 non-null  int64  
 8   origin_airport_id   55754 non-null  int64  
 9   origin              55754 non-null  object 
 10  origin_city_name    55754 non-null  object 
 11  dest_airport_id     55754 non-null  int64  
 12  dest                55754 non-null  object 
 13  dest_city_name      55754 non-null  object 
 14  crs_dep_time        55754 non-null  int64  
 15  crs_arr_time        55754 non-null  int64  
 16  dup 

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,arr_delay,target_class
0,2019-12-01,WN,WN,WN,3672,WN,N8315C,3672,10140,ABQ,...,BWI,"Baltimore, MD",1200,1735,N,215.0,1.0,1670.0,1.0,1
1,2019-12-01,WN,WN,WN,2772,WN,N431WN,2772,10397,ATL,...,BWI,"Baltimore, MD",1330,1510,N,100.0,1.0,577.0,10.0,1
2,2019-12-01,WN,WN,WN,5397,WN,N411WN,5397,10397,ATL,...,LAS,"Las Vegas, NV",740,915,N,275.0,1.0,1747.0,39.0,1


-----------------------------------------

#### __Import Summary Files__
> - Passenger Summary
> - Fuel Summary
> - Averave Arrival Delay

In [3]:
passengers = pd.read_csv('../passengers_summary.csv')
passengers.info()
passengers.head()

fuel = pd.read_csv('../fuel_summary.csv')
fuel.info()
fuel.head()

avg_arr = pd.read_csv('../arr_delay_avg_jan_dec.csv')
avg_arr.info()
avg_arr.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 139480 entries, 0 to 139479
Data columns (total 11 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   item_id           139480 non-null  object 
 1   carrier_month_id  139480 non-null  object 
 2   dest_airport_id   139480 non-null  int64  
 3   unique_carrier    139480 non-null  object 
 4   month             139480 non-null  int64  
 5   air_time          139480 non-null  float64
 6   distance          139480 non-null  float64
 7   passengers        139480 non-null  float64
 8   speed             139480 non-null  float64
 9   mean_passengers   139480 non-null  float64
 10  sum_passengers    139480 non-null  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 11.7+ MB


,item_id,carrier_month_id,dest_airport_id,unique_carrier,month,air_time,distance,passengers,speed,mean_passengers,sum_passengers
0,100055V10,5V10,10005,5V,10,954.0,1325.0,16.0,1.388889,10.0,8580
1,100145V10,5V10,10014,5V,10,82.0,70.0,0.0,0.853659,10.0,8580
2,100645V10,5V10,10064,5V,10,30.0,224.0,0.0,7.466667,10.0,8580
3,101395V10,5V10,10139,5V,10,91.0,323.0,0.0,3.549451,10.0,8580
4,101845V10,5V10,10184,5V,10,274.0,546.0,17.0,1.992701,10.0,8580


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 756 entries, 0 to 755
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   carrier_month_id    756 non-null    object 
 1   carrier             756 non-null    object 
 2   month               756 non-null    int64  
 3   total_cost:mean     756 non-null    float64
 4   total_cost:sum      756 non-null    float64
 5   total_gallons:mean  756 non-null    float64
 6   total_gallons:sum   756 non-null    float64
dtypes: float64(4), int64(1), object(2)
memory usage: 41.5+ KB


,carrier_month_id,carrier,month,total_cost:mean,total_cost:sum,total_gallons:mean,total_gallons:sum
0,09Q1,09Q,1,2.878729e+06,8636187.0,1.018414e+06,3055241.0
1,09Q10,09Q,10,2.892003e+06,8676010.0,1.210001e+06,3630002.0
2,09Q11,09Q,11,2.627896e+06,7883689.0,1.006282e+06,3018846.0
3,09Q12,09Q,12,2.743162e+06,8229487.0,9.321290e+05,2796387.0
4,09Q2,09Q,2,2.266957e+06,9067827.0,8.760805e+05,3504322.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 25968 entries, 0 to 25967
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   item_id    25968 non-null  object 
 1   arr_delay  25968 non-null  float64
dtypes: float64(1), object(1)
memory usage: 405.9+ KB


,item_id,arr_delay
0,9EABEATL1,7.073171
1,9EABEATL12,4.104762
2,9EABYATL1,12.512821
3,9EAEXATL1,-5.312195
4,9EAEXATL12,2.244344


-------------------------------------------

#### __Set Aside columns for creating the submission file later__

In [ ]:
df_submission_col = df[['fl_date', 'mkt_carrier', 'mkt_carrier_fl_num', 'origin', 'dest']]

df_submission_col.info()
df_submission_col.head()

# export to csv
df_submission_col.tp_csv('df_submission_columns.csv', index=False)

#### Step 1. Remove insignificant variables

In [4]:
# start time to measure the time of the program execution
start_time = time.time()

# drop columns
df = df.drop(columns=['branded_code_share', 'mkt_carrier', 
                      'origin_city_name', 'dest_city_name', 'dup', 
                      'origin_airport_id', 'tail_num',
                      'op_carrier_fl_num', 'target_class'
])

# rearrange columns
rearranged_columns = ['fl_date', 'dest_airport_id', 'mkt_unique_carrier', 'op_unique_carrier',
                      'mkt_carrier_fl_num', 'origin', 'dest', 'crs_dep_time', 'crs_arr_time', 
                      'crs_elapsed_time', 'flights', 'distance', 'arr_delay']

df = df[rearranged_columns]

# check
df.info()
df.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55754 entries, 0 to 55753
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fl_date             55754 non-null  object 
 1   dest_airport_id     55754 non-null  int64  
 2   mkt_unique_carrier  55754 non-null  object 
 3   op_unique_carrier   55754 non-null  object 
 4   mkt_carrier_fl_num  55754 non-null  int64  
 5   origin              55754 non-null  object 
 6   dest                55754 non-null  object 
 7   crs_dep_time        55754 non-null  int64  
 8   crs_arr_time        55754 non-null  int64  
 9   crs_elapsed_time    55754 non-null  float64
 10  flights             55754 non-null  float64
 11  distance            55754 non-null  float64
 12  arr_delay           55754 non-null  float64
dtypes: float64(4), int64(4), object(5)
memory usage: 5.5+ MB


,fl_date,dest_airport_id,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,flights,distance,arr_delay
0,2019-12-01,10821,WN,WN,3672,ABQ,BWI,1200,1735,215.0,1.0,1670.0,1.0
1,2019-12-01,10821,WN,WN,2772,ATL,BWI,1330,1510,100.0,1.0,577.0,10.0
2,2019-12-01,12889,WN,WN,5397,ATL,LAS,740,915,275.0,1.0,1747.0,39.0
3,2019-12-01,14492,WN,WN,5344,ATL,RDU,2245,10,85.0,1.0,356.0,23.0
4,2019-12-01,14635,WN,WN,3720,ATL,RSW,2040,2220,100.0,1.0,515.0,61.0




--- 0.07880783081054688 seconds ---


#### Step 2. Create additional features

In [5]:
# start time to measure the time of the program execution
start_time = time.time()

# convert 'fl_date' to datetime ('2019-05-01')
df['fl_date'] = pd.to_datetime(df['fl_date'], format='%Y-%m-%d')

# compute year, month and weekdays from 'fl_date' in df
month_train = [x.month for x in df['fl_date']]
year_train = [x.year for x in df['fl_date']]
dayofweek_train = df['fl_date'].dt.dayofweek

# insert new columns in df
df.insert(loc=1, column='year', value=year_train)
df.insert(loc=2, column='month', value=month_train)
df.insert(loc=3, column='day_of_week', value=dayofweek_train)

# check
df.info()
df.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55754 entries, 0 to 55753
Data columns (total 16 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   fl_date             55754 non-null  datetime64[ns]
 1   year                55754 non-null  int64         
 2   month               55754 non-null  int64         
 3   day_of_week         55754 non-null  int64         
 4   dest_airport_id     55754 non-null  int64         
 5   mkt_unique_carrier  55754 non-null  object        
 6   op_unique_carrier   55754 non-null  object        
 7   mkt_carrier_fl_num  55754 non-null  int64         
 8   origin              55754 non-null  object        
 9   dest                55754 non-null  object        
 10  crs_dep_time        55754 non-null  int64         
 11  crs_arr_time        55754 non-null  int64         
 12  crs_elapsed_time    55754 non-null  float64       
 13  flights             55754 non-null  float64   

,fl_date,year,month,day_of_week,dest_airport_id,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,flights,distance,arr_delay
0,2019-12-01,2019,12,6,10821,WN,WN,3672,ABQ,BWI,1200,1735,215.0,1.0,1670.0,1.0
1,2019-12-01,2019,12,6,10821,WN,WN,2772,ATL,BWI,1330,1510,100.0,1.0,577.0,10.0
2,2019-12-01,2019,12,6,12889,WN,WN,5397,ATL,LAS,740,915,275.0,1.0,1747.0,39.0
3,2019-12-01,2019,12,6,14492,WN,WN,5344,ATL,RDU,2245,10,85.0,1.0,356.0,23.0
4,2019-12-01,2019,12,6,14635,WN,WN,3720,ATL,RSW,2040,2220,100.0,1.0,515.0,61.0




--- 0.20956206321716309 seconds ---


#### Step 3. Get features from passengers, fuel & avgerage arrival delay

In [6]:
# start time to measure the time of the program execution
start_time = time.time()

# create new id variable to merge with passenger_summary table
item_id = []

for num in range(len(df)):
    item = str(df.iloc[num, 4]) + df.iloc[num, 6] + str(df.iloc[num, 2])
    item_id.append(item)

df.insert(loc=0, column='item_id', value=item_id)

# check
df.info()
df.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 55754 entries, 0 to 55753
Data columns (total 17 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   item_id             55754 non-null  object        
 1   fl_date             55754 non-null  datetime64[ns]
 2   year                55754 non-null  int64         
 3   month               55754 non-null  int64         
 4   day_of_week         55754 non-null  int64         
 5   dest_airport_id     55754 non-null  int64         
 6   mkt_unique_carrier  55754 non-null  object        
 7   op_unique_carrier   55754 non-null  object        
 8   mkt_carrier_fl_num  55754 non-null  int64         
 9   origin              55754 non-null  object        
 10  dest                55754 non-null  object        
 11  crs_dep_time        55754 non-null  int64         
 12  crs_arr_time        55754 non-null  int64         
 13  crs_elapsed_time    55754 non-null  float64   

,item_id,fl_date,year,month,day_of_week,dest_airport_id,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,flights,distance,arr_delay
0,10821WN12,2019-12-01,2019,12,6,10821,WN,WN,3672,ABQ,BWI,1200,1735,215.0,1.0,1670.0,1.0
1,10821WN12,2019-12-01,2019,12,6,10821,WN,WN,2772,ATL,BWI,1330,1510,100.0,1.0,577.0,10.0
2,12889WN12,2019-12-01,2019,12,6,12889,WN,WN,5397,ATL,LAS,740,915,275.0,1.0,1747.0,39.0
3,14492WN12,2019-12-01,2019,12,6,14492,WN,WN,5344,ATL,RDU,2245,10,85.0,1.0,356.0,23.0
4,14635WN12,2019-12-01,2019,12,6,14635,WN,WN,3720,ATL,RSW,2040,2220,100.0,1.0,515.0,61.0




--- 3.5540173053741455 seconds ---


In [7]:
# start time to measure the time of the program execution
start_time = time.time()

# remove columns from passenger_summary that should not be merged to df
pass_df = passengers.drop(columns=['carrier_month_id', 'dest_airport_id',
                                   'unique_carrier', 'month', 'air_time',
                                   'distance', 'passengers'])

# inner join sum_pass_df with carrier_pass
df = df.merge(pass_df, how='left', on='item_id')
df = df.rename({'speed': 'speed_passengers'}, axis=1)

# check
df.info()
df.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55754 entries, 0 to 55753
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   item_id             55754 non-null  object        
 1   fl_date             55754 non-null  datetime64[ns]
 2   year                55754 non-null  int64         
 3   month               55754 non-null  int64         
 4   day_of_week         55754 non-null  int64         
 5   dest_airport_id     55754 non-null  int64         
 6   mkt_unique_carrier  55754 non-null  object        
 7   op_unique_carrier   55754 non-null  object        
 8   mkt_carrier_fl_num  55754 non-null  int64         
 9   origin              55754 non-null  object        
 10  dest                55754 non-null  object        
 11  crs_dep_time        55754 non-null  int64         
 12  crs_arr_time        55754 non-null  int64         
 13  crs_elapsed_time    55754 non-null  float64   

,item_id,fl_date,year,month,day_of_week,dest_airport_id,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,flights,distance,arr_delay,speed_passengers,mean_passengers,sum_passengers
0,10821WN12,2019-12-01,2019,12,6,10821,WN,WN,3672,ABQ,BWI,1200,1735,215.0,1.0,1670.0,1.0,0.200252,12.0,184248
1,10821WN12,2019-12-01,2019,12,6,10821,WN,WN,2772,ATL,BWI,1330,1510,100.0,1.0,577.0,10.0,0.200252,12.0,184248
2,12889WN12,2019-12-01,2019,12,6,12889,WN,WN,5397,ATL,LAS,740,915,275.0,1.0,1747.0,39.0,0.198394,12.0,184248
3,14492WN12,2019-12-01,2019,12,6,14492,WN,WN,5344,ATL,RDU,2245,10,85.0,1.0,356.0,23.0,0.317143,12.0,184248
4,14635WN12,2019-12-01,2019,12,6,14635,WN,WN,3720,ATL,RSW,2040,2220,100.0,1.0,515.0,61.0,0.301315,12.0,184248




--- 0.11722469329833984 seconds ---


In [8]:
# start time to measure the time of the program execution
start_time = time.time()

# for three new columns merged from passenger_summary,
#  if there are any missing values, fill missing values with min values
df['speed_passengers'] = df['speed_passengers'].fillna(df['speed_passengers'].min())
df['mean_passengers'] = df['mean_passengers'].fillna(df['mean_passengers'].min())
df['sum_passengers'] = df['sum_passengers'].fillna(df['sum_passengers'].min())

# check
df.info()
df.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55754 entries, 0 to 55753
Data columns (total 20 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   item_id             55754 non-null  object        
 1   fl_date             55754 non-null  datetime64[ns]
 2   year                55754 non-null  int64         
 3   month               55754 non-null  int64         
 4   day_of_week         55754 non-null  int64         
 5   dest_airport_id     55754 non-null  int64         
 6   mkt_unique_carrier  55754 non-null  object        
 7   op_unique_carrier   55754 non-null  object        
 8   mkt_carrier_fl_num  55754 non-null  int64         
 9   origin              55754 non-null  object        
 10  dest                55754 non-null  object        
 11  crs_dep_time        55754 non-null  int64         
 12  crs_arr_time        55754 non-null  int64         
 13  crs_elapsed_time    55754 non-null  float64   

,item_id,fl_date,year,month,day_of_week,dest_airport_id,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,flights,distance,arr_delay,speed_passengers,mean_passengers,sum_passengers
0,10821WN12,2019-12-01,2019,12,6,10821,WN,WN,3672,ABQ,BWI,1200,1735,215.0,1.0,1670.0,1.0,0.200252,12.0,184248
1,10821WN12,2019-12-01,2019,12,6,10821,WN,WN,2772,ATL,BWI,1330,1510,100.0,1.0,577.0,10.0,0.200252,12.0,184248
2,12889WN12,2019-12-01,2019,12,6,12889,WN,WN,5397,ATL,LAS,740,915,275.0,1.0,1747.0,39.0,0.198394,12.0,184248
3,14492WN12,2019-12-01,2019,12,6,14492,WN,WN,5344,ATL,RDU,2245,10,85.0,1.0,356.0,23.0,0.317143,12.0,184248
4,14635WN12,2019-12-01,2019,12,6,14635,WN,WN,3720,ATL,RSW,2040,2220,100.0,1.0,515.0,61.0,0.301315,12.0,184248




--- 0.061524152755737305 seconds ---


In [9]:
# start time to measure the time of the program execution
start_time = time.time()

# create new id variable to merge with passenger_summary table
carrier_month_id = []

for num in range(len(df)):
    item = df.iloc[num, 7] + str(df.iloc[num, 3])
    carrier_month_id.append(item)

df.insert(loc=1, column='carrier_month_id', value=carrier_month_id)

# remove columns from fuel_summary that should not be merged to df
fuel_df = fuel.drop(columns=['carrier', 'month'])

# inner join sum_pass_df with carrier_pass
df = df.merge(fuel_df, how='left', on='carrier_month_id')
df = df.rename({'total_cost:mean': 'total_cost:mean_fuel',
               'total_cost:sum': 'total_cost:sum_fuel',
               'total_gallons:mean': 'total_gallons:mean_fuel',
               'total_gallons:sum': 'total_gallons:sum_fuel'}, axis=1)

# check
df.info()
df.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55754 entries, 0 to 55753
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   item_id                  55754 non-null  object        
 1   carrier_month_id         55754 non-null  object        
 2   fl_date                  55754 non-null  datetime64[ns]
 3   year                     55754 non-null  int64         
 4   month                    55754 non-null  int64         
 5   day_of_week              55754 non-null  int64         
 6   dest_airport_id          55754 non-null  int64         
 7   mkt_unique_carrier       55754 non-null  object        
 8   op_unique_carrier        55754 non-null  object        
 9   mkt_carrier_fl_num       55754 non-null  int64         
 10  origin                   55754 non-null  object        
 11  dest                     55754 non-null  object        
 12  crs_dep_time             55754 n

,item_id,carrier_month_id,fl_date,year,month,day_of_week,dest_airport_id,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,...,flights,distance,arr_delay,speed_passengers,mean_passengers,sum_passengers,total_cost:mean_fuel,total_cost:sum_fuel,total_gallons:mean_fuel,total_gallons:sum_fuel
0,10821WN12,WN12,2019-12-01,2019,12,6,10821,WN,WN,3672,...,1.0,1670.0,1.0,0.200252,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0
1,10821WN12,WN12,2019-12-01,2019,12,6,10821,WN,WN,2772,...,1.0,577.0,10.0,0.200252,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0
2,12889WN12,WN12,2019-12-01,2019,12,6,12889,WN,WN,5397,...,1.0,1747.0,39.0,0.198394,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0
3,14492WN12,WN12,2019-12-01,2019,12,6,14492,WN,WN,5344,...,1.0,356.0,23.0,0.317143,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0
4,14635WN12,WN12,2019-12-01,2019,12,6,14635,WN,WN,3720,...,1.0,515.0,61.0,0.301315,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0




--- 2.324354648590088 seconds ---


In [10]:
# for four new columns merged from fuel_summary,
#  if there are any missing values, fill missing values with 0
df['total_cost:mean_fuel'] = df['total_cost:mean_fuel'].fillna(df['total_cost:mean_fuel'].min())
df['total_cost:sum_fuel'] = df['total_cost:sum_fuel'].fillna(df['total_cost:sum_fuel'].min())
df['total_gallons:mean_fuel'] = df['total_gallons:mean_fuel'].fillna(df['total_gallons:mean_fuel'].min())
df['total_gallons:sum_fuel'] = df['total_gallons:sum_fuel'].fillna(df['total_gallons:sum_fuel'].min())

# check
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55754 entries, 0 to 55753
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   item_id                  55754 non-null  object        
 1   carrier_month_id         55754 non-null  object        
 2   fl_date                  55754 non-null  datetime64[ns]
 3   year                     55754 non-null  int64         
 4   month                    55754 non-null  int64         
 5   day_of_week              55754 non-null  int64         
 6   dest_airport_id          55754 non-null  int64         
 7   mkt_unique_carrier       55754 non-null  object        
 8   op_unique_carrier        55754 non-null  object        
 9   mkt_carrier_fl_num       55754 non-null  int64         
 10  origin                   55754 non-null  object        
 11  dest                     55754 non-null  object        
 12  crs_dep_time             55754 n

,item_id,carrier_month_id,fl_date,year,month,day_of_week,dest_airport_id,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,...,flights,distance,arr_delay,speed_passengers,mean_passengers,sum_passengers,total_cost:mean_fuel,total_cost:sum_fuel,total_gallons:mean_fuel,total_gallons:sum_fuel
0,10821WN12,WN12,2019-12-01,2019,12,6,10821,WN,WN,3672,...,1.0,1670.0,1.0,0.200252,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0
1,10821WN12,WN12,2019-12-01,2019,12,6,10821,WN,WN,2772,...,1.0,577.0,10.0,0.200252,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0
2,12889WN12,WN12,2019-12-01,2019,12,6,12889,WN,WN,5397,...,1.0,1747.0,39.0,0.198394,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0
3,14492WN12,WN12,2019-12-01,2019,12,6,14492,WN,WN,5344,...,1.0,356.0,23.0,0.317143,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0
4,14635WN12,WN12,2019-12-01,2019,12,6,14635,WN,WN,3720,...,1.0,515.0,61.0,0.301315,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0


In [11]:
# start time to measure the time of the program execution
start_time = time.time()

# remove the existing item_id from df
df = df.drop(columns=['item_id'])

# create new id variable to merge with passenger_summary table
item_id = []

for num in range(len(df)):
    item = df.iloc[num, 7] + df.iloc[num, 9] + df.iloc[num, 10] + str(df.iloc[num, 3])
    item_id.append(item)

df.insert(loc=0, column='item_id', value=item_id)

# check
df.info()
df.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55754 entries, 0 to 55753
Data columns (total 25 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   item_id                  55754 non-null  object        
 1   carrier_month_id         55754 non-null  object        
 2   fl_date                  55754 non-null  datetime64[ns]
 3   year                     55754 non-null  int64         
 4   month                    55754 non-null  int64         
 5   day_of_week              55754 non-null  int64         
 6   dest_airport_id          55754 non-null  int64         
 7   mkt_unique_carrier       55754 non-null  object        
 8   op_unique_carrier        55754 non-null  object        
 9   mkt_carrier_fl_num       55754 non-null  int64         
 10  origin                   55754 non-null  object        
 11  dest                     55754 non-null  object        
 12  crs_dep_time             55754 n

,item_id,carrier_month_id,fl_date,year,month,day_of_week,dest_airport_id,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,...,flights,distance,arr_delay,speed_passengers,mean_passengers,sum_passengers,total_cost:mean_fuel,total_cost:sum_fuel,total_gallons:mean_fuel,total_gallons:sum_fuel
0,WNABQBWI12,WN12,2019-12-01,2019,12,6,10821,WN,WN,3672,...,1.0,1670.0,1.0,0.200252,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0
1,WNATLBWI12,WN12,2019-12-01,2019,12,6,10821,WN,WN,2772,...,1.0,577.0,10.0,0.200252,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0
2,WNATLLAS12,WN12,2019-12-01,2019,12,6,12889,WN,WN,5397,...,1.0,1747.0,39.0,0.198394,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0
3,WNATLRDU12,WN12,2019-12-01,2019,12,6,14492,WN,WN,5344,...,1.0,356.0,23.0,0.317143,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0
4,WNATLRSW12,WN12,2019-12-01,2019,12,6,14635,WN,WN,3720,...,1.0,515.0,61.0,0.301315,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0




--- 4.257476568222046 seconds ---


In [12]:
# start time to measure the time of the program execution
start_time = time.time()

avg_arr = avg_arr.rename({'arr_delay': 'avg_arr_delay'}, axis=1)

# inner join sum_pass_df with carrier_pass
df = df.merge(avg_arr, how='left', on='item_id')

# check
df.info()
df.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55754 entries, 0 to 55753
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   item_id                  55754 non-null  object        
 1   carrier_month_id         55754 non-null  object        
 2   fl_date                  55754 non-null  datetime64[ns]
 3   year                     55754 non-null  int64         
 4   month                    55754 non-null  int64         
 5   day_of_week              55754 non-null  int64         
 6   dest_airport_id          55754 non-null  int64         
 7   mkt_unique_carrier       55754 non-null  object        
 8   op_unique_carrier        55754 non-null  object        
 9   mkt_carrier_fl_num       55754 non-null  int64         
 10  origin                   55754 non-null  object        
 11  dest                     55754 non-null  object        
 12  crs_dep_time             55754 n

,item_id,carrier_month_id,fl_date,year,month,day_of_week,dest_airport_id,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,...,distance,arr_delay,speed_passengers,mean_passengers,sum_passengers,total_cost:mean_fuel,total_cost:sum_fuel,total_gallons:mean_fuel,total_gallons:sum_fuel,avg_arr_delay
0,WNABQBWI12,WN12,2019-12-01,2019,12,6,10821,WN,WN,3672,...,1670.0,1.0,0.200252,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0,-5.156250
1,WNATLBWI12,WN12,2019-12-01,2019,12,6,10821,WN,WN,2772,...,577.0,10.0,0.200252,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0,5.637982
2,WNATLLAS12,WN12,2019-12-01,2019,12,6,12889,WN,WN,5397,...,1747.0,39.0,0.198394,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0,2.584416
3,WNATLRDU12,WN12,2019-12-01,2019,12,6,14492,WN,WN,5344,...,356.0,23.0,0.317143,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0,10.407143
4,WNATLRSW12,WN12,2019-12-01,2019,12,6,14635,WN,WN,3720,...,515.0,61.0,0.301315,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0,15.902174




--- 0.11439228057861328 seconds ---


In [13]:
# start time to measure the time of the program execution
start_time = time.time()

# for three new columns merged from passenger_summary,
#  if there are any missing values, fill missing values with min values
df['avg_arr_delay'] = df['avg_arr_delay'].fillna(df['avg_arr_delay'].min())

# check
df.info()
df.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55754 entries, 0 to 55753
Data columns (total 26 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   item_id                  55754 non-null  object        
 1   carrier_month_id         55754 non-null  object        
 2   fl_date                  55754 non-null  datetime64[ns]
 3   year                     55754 non-null  int64         
 4   month                    55754 non-null  int64         
 5   day_of_week              55754 non-null  int64         
 6   dest_airport_id          55754 non-null  int64         
 7   mkt_unique_carrier       55754 non-null  object        
 8   op_unique_carrier        55754 non-null  object        
 9   mkt_carrier_fl_num       55754 non-null  int64         
 10  origin                   55754 non-null  object        
 11  dest                     55754 non-null  object        
 12  crs_dep_time             55754 n

,item_id,carrier_month_id,fl_date,year,month,day_of_week,dest_airport_id,mkt_unique_carrier,op_unique_carrier,mkt_carrier_fl_num,...,distance,arr_delay,speed_passengers,mean_passengers,sum_passengers,total_cost:mean_fuel,total_cost:sum_fuel,total_gallons:mean_fuel,total_gallons:sum_fuel,avg_arr_delay
0,WNABQBWI12,WN12,2019-12-01,2019,12,6,10821,WN,WN,3672,...,1670.0,1.0,0.200252,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0,-5.156250
1,WNATLBWI12,WN12,2019-12-01,2019,12,6,10821,WN,WN,2772,...,577.0,10.0,0.200252,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0,5.637982
2,WNATLLAS12,WN12,2019-12-01,2019,12,6,12889,WN,WN,5397,...,1747.0,39.0,0.198394,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0,2.584416
3,WNATLRDU12,WN12,2019-12-01,2019,12,6,14492,WN,WN,5344,...,356.0,23.0,0.317143,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0,10.407143
4,WNATLRSW12,WN12,2019-12-01,2019,12,6,14635,WN,WN,3720,...,515.0,61.0,0.301315,12.0,184248,309959717.0,1.239839e+09,170009681.0,680038724.0,15.902174




--- 0.07126784324645996 seconds ---


#### Setp 4. Create dummy variables based on the following features
> - year
> - month
> - day_of_week
> - mkt_unique_carrier & op_unique_carrier

In [14]:
# start time to measure the time of the program execution
start_time = time.time()

df['year'] = df['year'].astype(str)
df['month'] = df['month'].astype(str)
df['day_of_week'] = df['day_of_week'].astype(str)

# create dummy variables
dummy_features = ['year', 'month', 'day_of_week', 'mkt_unique_carrier', 'op_unique_carrier']

df_dummy = pd.get_dummies(df[dummy_features])
df_dummy

# remove variables related to dummy_features from work_df
df = df.drop(dummy_features, axis=1)

# combine dummy variables with work_df
df = pd.concat([df, df_dummy], axis=1)

# check
df.info()
df.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

,year_2018,year_2019,month_1,month_12,day_of_week_0,day_of_week_1,day_of_week_2,day_of_week_3,day_of_week_4,day_of_week_5,...,op_unique_carrier_OH,op_unique_carrier_OO,op_unique_carrier_PT,op_unique_carrier_QX,op_unique_carrier_UA,op_unique_carrier_VX,op_unique_carrier_WN,op_unique_carrier_YV,op_unique_carrier_YX,op_unique_carrier_ZW
0,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
1,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
3,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
4,0,1,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55749,1,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55750,1,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55751,1,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
55752,1,0,1,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


<class 'pandas.core.frame.DataFrame'>
Int64Index: 55754 entries, 0 to 55753
Data columns (total 71 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   item_id                  55754 non-null  object        
 1   carrier_month_id         55754 non-null  object        
 2   fl_date                  55754 non-null  datetime64[ns]
 3   dest_airport_id          55754 non-null  int64         
 4   mkt_carrier_fl_num       55754 non-null  int64         
 5   origin                   55754 non-null  object        
 6   dest                     55754 non-null  object        
 7   crs_dep_time             55754 non-null  int64         
 8   crs_arr_time             55754 non-null  int64         
 9   crs_elapsed_time         55754 non-null  float64       
 10  flights                  55754 non-null  float64       
 11  distance                 55754 non-null  float64       
 12  arr_delay                55754 n

,item_id,carrier_month_id,fl_date,dest_airport_id,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,...,op_unique_carrier_OH,op_unique_carrier_OO,op_unique_carrier_PT,op_unique_carrier_QX,op_unique_carrier_UA,op_unique_carrier_VX,op_unique_carrier_WN,op_unique_carrier_YV,op_unique_carrier_YX,op_unique_carrier_ZW
0,WNABQBWI12,WN12,2019-12-01,10821,3672,ABQ,BWI,1200,1735,215.0,...,0,0,0,0,0,0,1,0,0,0
1,WNATLBWI12,WN12,2019-12-01,10821,2772,ATL,BWI,1330,1510,100.0,...,0,0,0,0,0,0,1,0,0,0
2,WNATLLAS12,WN12,2019-12-01,12889,5397,ATL,LAS,740,915,275.0,...,0,0,0,0,0,0,1,0,0,0
3,WNATLRDU12,WN12,2019-12-01,14492,5344,ATL,RDU,2245,10,85.0,...,0,0,0,0,0,0,1,0,0,0
4,WNATLRSW12,WN12,2019-12-01,14635,3720,ATL,RSW,2040,2220,100.0,...,0,0,0,0,0,0,1,0,0,0




--- 0.20445680618286133 seconds ---


In [15]:
# start time to measure the time of the program execution
start_time = time.time()

# remove variables related to dummy_features from work_df
df = df.drop(['item_id', 'carrier_month_id'], axis=1)

# check
df.info()
df.head(3)

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55754 entries, 0 to 55753
Data columns (total 69 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   fl_date                  55754 non-null  datetime64[ns]
 1   dest_airport_id          55754 non-null  int64         
 2   mkt_carrier_fl_num       55754 non-null  int64         
 3   origin                   55754 non-null  object        
 4   dest                     55754 non-null  object        
 5   crs_dep_time             55754 non-null  int64         
 6   crs_arr_time             55754 non-null  int64         
 7   crs_elapsed_time         55754 non-null  float64       
 8   flights                  55754 non-null  float64       
 9   distance                 55754 non-null  float64       
 10  arr_delay                55754 non-null  float64       
 11  speed_passengers         55754 non-null  float64       
 12  mean_passengers          55754 n

,fl_date,dest_airport_id,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,flights,distance,...,op_unique_carrier_OH,op_unique_carrier_OO,op_unique_carrier_PT,op_unique_carrier_QX,op_unique_carrier_UA,op_unique_carrier_VX,op_unique_carrier_WN,op_unique_carrier_YV,op_unique_carrier_YX,op_unique_carrier_ZW
0,2019-12-01,10821,3672,ABQ,BWI,1200,1735,215.0,1.0,1670.0,...,0,0,0,0,0,0,1,0,0,0
1,2019-12-01,10821,2772,ATL,BWI,1330,1510,100.0,1.0,577.0,...,0,0,0,0,0,0,1,0,0,0
2,2019-12-01,12889,5397,ATL,LAS,740,915,275.0,1.0,1747.0,...,0,0,0,0,0,0,1,0,0,0




--- 0.05870366096496582 seconds ---


#### __Create Ordinal Variables based on Flights__
> - origin
> - dest

#### __origin__

In [16]:
# start time to measure the time of the program execution
start_time = time.time()

# create summary table of tail_num vs flights
origin_table = pd.pivot_table(df, values='flights', index=['origin'],
                                aggfunc=np.sum)

# reindex the rows to restore the variable in index to column
origin_table = origin_table.reset_index(drop=False)
origin_table

# get min & Max values from the table
min_1 = min(origin_table['flights'])
max_1 = max(origin_table['flights'])
#bin_size1 = int((max_1 - min_1) / 5 + 1)

print(f"Max Value: {max_1} | Min Value: {min_1}")

# allocate bin_size to 5 groups
#group11 = 1000
#group21 = 2000
#group31 = 5000
#group41 = 11000
#group51 = 27000
#group11 = int(min_1 + bin_size1)
#group21 = int(group11 + bin_size1)
#group31 = int(group21 + bin_size1)
#group41 = int(group31 + bin_size1)
#group51 = int(group41 + bin_size1)

#print(f"group 1: {group11} | group 2: {group21} | group 3: {group31} | \
#group 4: {group41} | group 5: {group51}")

# assign the group number to tail_num values
origin_class = []

#for value in origin_table['flights']:
#    if value <= group11:
#        origin_class.append(1)
#    elif value > group11 and value <= group21:
#        origin_class.append(2)
#    elif value > group21 and value <= group31:
#        origin_class.append(3)
#    elif value > group31 and value <= group41:
#        origin_class.append(4)
#    elif value > group41 and value <= group51:
#        origin_class.append(5)

# assign the list above to a new class variable
#origin_table['class'] = origin_class
origin_table['class'] = origin_table['flights'].astype(int)

# check 
origin_table
origin_table['class'].nunique()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

,origin,flights
0,ABE,40.0
1,ABI,18.0
2,ABQ,186.0
3,ABR,7.0
4,ABY,9.0
...,...,...
357,XNA,122.0
358,XWA,5.0
359,YAK,4.0
360,YKM,7.0


Max Value: 2716.0 | Min Value: 1.0


,origin,flights,class
0,ABE,40.0,40
1,ABI,18.0,18
2,ABQ,186.0,186
3,ABR,7.0,7
4,ABY,9.0,9
...,...,...,...
357,XNA,122.0,122
358,XWA,5.0,5
359,YAK,4.0,4
360,YKM,7.0,7


156



--- 0.03672385215759277 seconds ---


In [17]:
# start time to measure the time of the program execution
start_time = time.time()

# create a dictiornary - key with tail_num and value with class
# restructure the tableto prepare for a dictionary
origin_df = origin_table.drop(['flights'], axis=1)
origin_df

# create an empty dictionary
dict1 = {}

for num in range(len(origin_df)):
    dict1[origin_df.iloc[num, 0]] = origin_df.iloc[num, 1]

# check dictionary created from above
#dict1

# map the ordinal features, using the dictionary above
df = df.replace({"origin" : dict1})

# check
df.info()
df.head(3)

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

,origin,class
0,ABE,40
1,ABI,18
2,ABQ,186
3,ABR,7
4,ABY,9
...,...,...
357,XNA,122
358,XWA,5
359,YAK,4
360,YKM,7


<class 'pandas.core.frame.DataFrame'>
Int64Index: 55754 entries, 0 to 55753
Data columns (total 69 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   fl_date                  55754 non-null  datetime64[ns]
 1   dest_airport_id          55754 non-null  int64         
 2   mkt_carrier_fl_num       55754 non-null  int64         
 3   origin                   55754 non-null  int32         
 4   dest                     55754 non-null  object        
 5   crs_dep_time             55754 non-null  int64         
 6   crs_arr_time             55754 non-null  int64         
 7   crs_elapsed_time         55754 non-null  float64       
 8   flights                  55754 non-null  float64       
 9   distance                 55754 non-null  float64       
 10  arr_delay                55754 non-null  float64       
 11  speed_passengers         55754 non-null  float64       
 12  mean_passengers          55754 n

,fl_date,dest_airport_id,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,flights,distance,...,op_unique_carrier_OH,op_unique_carrier_OO,op_unique_carrier_PT,op_unique_carrier_QX,op_unique_carrier_UA,op_unique_carrier_VX,op_unique_carrier_WN,op_unique_carrier_YV,op_unique_carrier_YX,op_unique_carrier_ZW
0,2019-12-01,10821,3672,186,BWI,1200,1735,215.0,1.0,1670.0,...,0,0,0,0,0,0,1,0,0,0
1,2019-12-01,10821,2772,2683,BWI,1330,1510,100.0,1.0,577.0,...,0,0,0,0,0,0,1,0,0,0
2,2019-12-01,12889,5397,2683,LAS,740,915,275.0,1.0,1747.0,...,0,0,0,0,0,0,1,0,0,0




--- 0.6092584133148193 seconds ---


In [18]:
df['origin'].unique()

array([ 186, 2683, 1147, 2049,  318,   18,  627, 1059, 1089,  468, 1116,
       1143,   46, 1305,  127, 1272,  577,  123,  148, 1829,  132,  356,
        208,  376, 2716,   65,   53, 1185,  201,  176,  819,  724,  348,
        708,  157,  999,  422, 1251,  989,   47,  843,  864, 1399, 1965,
        383, 1819,  645,  179,  672,  441,  211,  513,  212,  168,  227,
       1011,  408,  602,   76,  262,  365,  257,  545,  231,  121,   12,
         45,   28,  165,  117,   40,   22,  139,   99,  108,   77,   10,
         43,  467,   15,    8,   56,   54,   19,  206,   48,   68,   67,
         32,  310,  190,   39,  122,   72,  171,  338,  306,  130,  118,
         26,   81,   44,   37,  414,   55,    5,   58,   69,   64,  131,
         35,  105,   94,   90,    6,   49,   62,  525,   16,  223,    4,
        163,   89,   63,   31,   34,   30,   59,   27,   61,   52,  142,
         13,   23,    7,   33,   71,   36,   17,  102,   91,    1,   20,
         24,   50,   41,   29,   14,   21,   38,   

#### __dest__

In [19]:
# start time to measure the time of the program execution
start_time = time.time()

# create summary table of tail_num vs flights
dest_table = pd.pivot_table(df, values='flights', index=['dest'],
                                aggfunc=np.sum)

# reindex the rows to restore the variable in index to column
dest_table = dest_table.reset_index(drop=False)
dest_table

# get min & Max values from the table
min_3 = min(dest_table['flights'])
max_3 = max(dest_table['flights'])
#bin_size3 = int((max_tailnum3 - min_tailnum3) / 5 + 1)

print(f"Max Value: {max_3} | Min Value: {min_3}")

# allocate bin_size to 5 groups
#group13 = 1000
#group23 = 2000
#group33 = 6000
#group43 = 12000
#group53 = 26000

#print(f"group 1: {group13} | group 2: {group23} | group 3: {group33} | \
#group 4: {group43} | group 5: {group53}")

# assign the group number to tail_num values
#dest_class = []

#for value in dest_table['flights']:
#    if value <= group13:
#        dest_class.append(1)
#    elif value > group13 and value <= group23:
#        dest_class.append(2)
#    elif value > group23 and value <= group33:
#        dest_class.append(3)
#    elif value > group33 and value <= group43:
#        dest_class.append(4)
#    elif value > group43 and value <= group53:
#        dest_class.append(5)

# assign the list above to a new class variable
#dest_table['class'] = dest_class
dest_table['class'] = dest_table['flights'].astype(int)

# check 
dest_table
dest_table['class'].nunique()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

,dest,flights
0,ABE,38.0
1,ABI,10.0
2,ABQ,203.0
3,ABR,6.0
4,ABY,5.0
...,...,...
352,XNA,86.0
353,XWA,4.0
354,YAK,3.0
355,YKM,13.0


Max Value: 2708.0 | Min Value: 1.0


,dest,flights,class
0,ABE,38.0,38
1,ABI,10.0,10
2,ABQ,203.0,203
3,ABR,6.0,6
4,ABY,5.0,5
...,...,...,...
352,XNA,86.0,86
353,XWA,4.0,4
354,YAK,3.0,3
355,YKM,13.0,13


159



--- 0.02976822853088379 seconds ---


In [20]:
# start time to measure the time of the program execution
start_time = time.time()

# create a dictiornary - key with tail_num and value with class
# restructure the tableto prepare for a dictionary
dest_df = dest_table.drop(['flights'], axis=1)
dest_df

# create an empty dictionary
dict3 = {}

for num in range(len(dest_df)):
    dict3[dest_df.iloc[num, 0]] = dest_df.iloc[num, 1]

# check dictionary created from above
#dict3

# map the ordinal features, using the dictionary above
df = df.replace({"dest" : dict3})

# check
df.info()
df.head(3)

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

,dest,class
0,ABE,38
1,ABI,10
2,ABQ,203
3,ABR,6
4,ABY,5
...,...,...
352,XNA,86
353,XWA,4
354,YAK,3
355,YKM,13


<class 'pandas.core.frame.DataFrame'>
Int64Index: 55754 entries, 0 to 55753
Data columns (total 69 columns):
 #   Column                   Non-Null Count  Dtype         
---  ------                   --------------  -----         
 0   fl_date                  55754 non-null  datetime64[ns]
 1   dest_airport_id          55754 non-null  int64         
 2   mkt_carrier_fl_num       55754 non-null  int64         
 3   origin                   55754 non-null  int32         
 4   dest                     55754 non-null  int32         
 5   crs_dep_time             55754 non-null  int64         
 6   crs_arr_time             55754 non-null  int64         
 7   crs_elapsed_time         55754 non-null  float64       
 8   flights                  55754 non-null  float64       
 9   distance                 55754 non-null  float64       
 10  arr_delay                55754 non-null  float64       
 11  speed_passengers         55754 non-null  float64       
 12  mean_passengers          55754 n

,fl_date,dest_airport_id,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,flights,distance,...,op_unique_carrier_OH,op_unique_carrier_OO,op_unique_carrier_PT,op_unique_carrier_QX,op_unique_carrier_UA,op_unique_carrier_VX,op_unique_carrier_WN,op_unique_carrier_YV,op_unique_carrier_YX,op_unique_carrier_ZW
0,2019-12-01,10821,3672,186,765,1200,1735,215.0,1.0,1670.0,...,0,0,0,0,0,0,1,0,0,0
1,2019-12-01,10821,2772,2683,765,1330,1510,100.0,1.0,577.0,...,0,0,0,0,0,0,1,0,0,0
2,2019-12-01,12889,5397,2683,1137,740,915,275.0,1.0,1747.0,...,0,0,0,0,0,0,1,0,0,0




--- 0.6564795970916748 seconds ---


In [21]:
df['dest'].unique()

array([ 765, 1137,  459,  324,  681, 1856,  127, 1026,  255,  235,  380,
       1247,  131,  927, 1329, 2020,  606,  136,  405,   85,  158, 1727,
         53, 1067,  175, 1057,  344,  345, 1167, 2708,  436,  634, 1810,
       1209,  387,  467, 1066,   79,  328,  875, 1144, 2663,   72,  689,
         23,  546, 1262, 1078,  236,  787,  398,  173,  522,  141,  297,
       1286,  567,  160,  138,  437,  462,  178,  162,  222,   54,   14,
         32,   42,  320,   55,   30,    6,   89,  153,   73,  420,  106,
         86,   66,   56,   78,   96,  139,    4,   37,   19,   35,   59,
        135,  125,  544,   20,   48,  341,   27,   80,   15,   26,   45,
        159,  161,   49,   12,   69,   17,   24,   31,   40,  229,   94,
         65,  113,   11,  203,  184,  157,  165,    9,  115,   10,  156,
         84,   88,   38,   18,   51,    5,    7,   68,   76,   57,   39,
         22,    8,   61,   62,  190,   28,   71,   29,   33,  130,  102,
         25,  112,   95,   58,   36,   21,   34,   

#### __Remove 'fl_date', 'flights' & 'dest_airport_id' from df__

In [22]:
# start time to measure the time of the program execution
start_time = time.time()

# drop columns
df = df.drop(columns=['fl_date', 'dest_airport_id', 'flights'])

# check
df.info()
df.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55754 entries, 0 to 55753
Data columns (total 66 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mkt_carrier_fl_num       55754 non-null  int64  
 1   origin                   55754 non-null  int32  
 2   dest                     55754 non-null  int32  
 3   crs_dep_time             55754 non-null  int64  
 4   crs_arr_time             55754 non-null  int64  
 5   crs_elapsed_time         55754 non-null  float64
 6   distance                 55754 non-null  float64
 7   arr_delay                55754 non-null  float64
 8   speed_passengers         55754 non-null  float64
 9   mean_passengers          55754 non-null  float64
 10  sum_passengers           55754 non-null  int64  
 11  total_cost:mean_fuel     55754 non-null  float64
 12  total_cost:sum_fuel      55754 non-null  float64
 13  total_gallons:mean_fuel  55754 non-null  float64
 14  total_gallons:sum_fuel

,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,speed_passengers,mean_passengers,...,op_unique_carrier_OH,op_unique_carrier_OO,op_unique_carrier_PT,op_unique_carrier_QX,op_unique_carrier_UA,op_unique_carrier_VX,op_unique_carrier_WN,op_unique_carrier_YV,op_unique_carrier_YX,op_unique_carrier_ZW
0,3672,186,765,1200,1735,215.0,1670.0,1.0,0.200252,12.0,...,0,0,0,0,0,0,1,0,0,0
1,2772,2683,765,1330,1510,100.0,577.0,10.0,0.200252,12.0,...,0,0,0,0,0,0,1,0,0,0
2,5397,2683,1137,740,915,275.0,1747.0,39.0,0.198394,12.0,...,0,0,0,0,0,0,1,0,0,0
3,5344,2683,459,2245,10,85.0,356.0,23.0,0.317143,12.0,...,0,0,0,0,0,0,1,0,0,0
4,3720,2683,324,2040,2220,100.0,515.0,61.0,0.301315,12.0,...,0,0,0,0,0,0,1,0,0,0




--- 0.03399395942687988 seconds ---


--------------------------------------------

#### __Only for Training Dataset: Skip this process for Testing Dataset__

In [23]:
# start time to measure the time of the program execution
start_time = time.time()

# Move 'target_class' variable to the last column
target = df['arr_delay']
target

# remove variables related to dummy_features from work_df
df = df.drop(['arr_delay'], axis=1)

# add back the target variable to the last column'
df['target'] = target

# check
df.info()
df.head()

# print the overall program runtime.
print(f"\n\n--- {(time.time() - start_time)} seconds ---")

0          1.0
1         10.0
2         39.0
3         23.0
4         61.0
         ...  
55749   -124.0
55750     -1.0
55751   -230.0
55752    -32.0
55753   -166.0
Name: arr_delay, Length: 55754, dtype: float64

<class 'pandas.core.frame.DataFrame'>
Int64Index: 55754 entries, 0 to 55753
Data columns (total 66 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   mkt_carrier_fl_num       55754 non-null  int64  
 1   origin                   55754 non-null  int32  
 2   dest                     55754 non-null  int32  
 3   crs_dep_time             55754 non-null  int64  
 4   crs_arr_time             55754 non-null  int64  
 5   crs_elapsed_time         55754 non-null  float64
 6   distance                 55754 non-null  float64
 7   speed_passengers         55754 non-null  float64
 8   mean_passengers          55754 non-null  float64
 9   sum_passengers           55754 non-null  int64  
 10  total_cost:mean_fuel     55754 non-null  float64
 11  total_cost:sum_fuel      55754 non-null  float64
 12  total_gallons:mean_fuel  55754 non-null  float64
 13  total_gallons:sum_fuel   55754 non-null  float64
 14  avg_arr_delay         

,mkt_carrier_fl_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,speed_passengers,mean_passengers,sum_passengers,...,op_unique_carrier_OO,op_unique_carrier_PT,op_unique_carrier_QX,op_unique_carrier_UA,op_unique_carrier_VX,op_unique_carrier_WN,op_unique_carrier_YV,op_unique_carrier_YX,op_unique_carrier_ZW,target
0,3672,186,765,1200,1735,215.0,1670.0,0.200252,12.0,184248,...,0,0,0,0,0,1,0,0,0,1.0
1,2772,2683,765,1330,1510,100.0,577.0,0.200252,12.0,184248,...,0,0,0,0,0,1,0,0,0,10.0
2,5397,2683,1137,740,915,275.0,1747.0,0.198394,12.0,184248,...,0,0,0,0,0,1,0,0,0,39.0
3,5344,2683,459,2245,10,85.0,356.0,0.317143,12.0,184248,...,0,0,0,0,0,1,0,0,0,23.0
4,3720,2683,324,2040,2220,100.0,515.0,0.301315,12.0,184248,...,0,0,0,0,0,1,0,0,0,61.0




--- 0.0639798641204834 seconds ---


-------------------------------------------------------

In [24]:
# export to csv
df.to_csv('training_for_regmodel_56k.csv', index=False)